# STEP 0: Set up

To get started on this notebook, you’ll need to restore any variables
from previous notebooks to your workspace. To save time and memory, make
sure to specify which variables you want to load.

In [1]:
%store -r ndvi_da gdf

You will also need to import any libraries you are using in this
notebook, since they won’t carry over from the previous notebook:

In [2]:
# Import libraries

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import rioxarray as rxr
import xarray as xr

import earthpy
import glob
import hvplot.pandas
import hvplot.xarray

from shapely.geometry import mapping

# STEP 4: Is the NDVI different within the **?meta:params.site_name** after the **?meta:params.event**?

You will compute the mean NDVI inside and outside the fire boundary.
First, use the code below to get a `GeoDataFrame` of the area outside
the Reservation.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Check the variable names - Make sure that the code uses your
boundary <code>GeoDataFrame</code></li>
<li>How could you test if the geometry was modified correctly? Add some
code to take a look at the results.</li>
</ol></div></div>

In [3]:
# Check CRSs, make them match if needed.

print(ndvi_da.NDVI.rio.crs) # EPSG:4326
print(gdf.crs) # EPSG:4269

gdf.to_crs('EPSG:4326', inplace=True)
print("the gdf CRS is now:", gdf.crs)

EPSG:4326
EPSG:4269
the gdf CRS is now: EPSG:4326


Next, clip your DataArray to the boundaries for both inside and outside
the reservation. You will need to replace the `GeoDataFrame` name with
your own. Check out the [lesson on clipping data with the `rioxarray`
library in the
textbook](https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/raster-data-processing/crop-raster-data-with-shapefile-in-python/).

> **GOTCHA ALERT**
>
> It’s important to use `from_disk=True` when clipping large arrays like
> this. It allows the computer to use less valuable memory resources
> when clipping - you will probably find that otherwise the cell below
> crashes your kernel.

In [4]:
# Clip data to both inside and outside the boundary

# Inner
ndvi_in = ndvi_da.rio.clip(gdf.geometry.apply(mapping))

# Outer
ndvi_out = ndvi_da.rio.clip(
    gdf.geometry.apply(mapping), invert=True
    )

In [5]:
# Quick plots
# We should see the two plots match the clipped area like puzzle pieces.

# Make plots
plot_in = ndvi_in.isel(date=0, band=0).hvplot(x='x', y='y', cmap=plt.cm.PiYG, geo=True, 
                   title='NDVI inside Tribal Boundary')

plot_out = ndvi_out.isel(date=0, band=0).hvplot(x='x', y='y', cmap=plt.cm.PiYG, geo=True, 
                   title='NDVI outside Tribal Boundary')

# Show plots
(plot_in + plot_out)

:Layout
   .Image.I  :Image   [x,y]   (NDVI)
   .Image.II :Image   [x,y]   (NDVI)

The plot above shows that I've successfully clipped the NDVI array into arrays inside and outside the tribal boundary! You can see that the inner plot shows NDVI values inside the boundary, and the outer plot shows NDVI values outside. Now we're ready to calculate mean values over time.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>For <strong>both inside and outside</strong> the <span
data-__quarto_custom="true" data-__quarto_custom_type="Shortcode"
data-__quarto_custom_context="Inline"
data-__quarto_custom_id="3"></span> boundary:</p>
<ul>
<li>Group the data by year</li>
<li>Take the mean. You always need to tell reducing methods in
<code>xarray</code> what dimensions you want to reduce. When you want to
summarize data across <strong>all</strong> dimensions, you can use the
<code>...</code> syntax, e.g. <code>.mean(...)</code> as a
shorthand.</li>
<li>Select the NDVI variable</li>
<li>Convert to a DataFrame using the <code>to_dataframe()</code>
method</li>
<li>Join the two DataFrames for plotting using the <code>.join()</code>
method. You will need to rename the columns using the
<code>lsuffix=</code> and <code>rsuffix=</code> parameters</li>
</ul>
<p>Finally, plot annual July means for both inside and outside the
Reservation on the same plot.</p></div></div>

> **GOTCHA ALERT**
>
> The DateIndex in pandas is a little different from the Datetime
> Dimension in xarray. You will need to use the `.dt.year` syntax to
> access information about the year, not just `.year`.

#### Based on the explicit bullet point instructions, we're just taking the annual mean, not the July means. I''ve run into a bunch of trouble trying to get annual July means (it seems like there's no July data after 2005), so I dropped the monthly component and now things work as expected. My graphs match what's shown on the website, so I'm sticking with just annual means. Unfortunate that things were confusing in the instructions here.

In [6]:
# Compute mean annual NDVI (not july - this matches video and website)
# I had miscounted the dates in part 2, but have fixed that and this works now.

# Inside
ndvi_in_mean_df = (ndvi_in
    
    #.sel(date=ndvi_in['date'].dt.month.isin([7])) 
    # Select NDVI data in July; I've commented this out so the 
    # analysis works

    # Now group into years
    .groupby('date.year')

    # Now take the mean across all dimensions
    .mean(...)
    .to_dataframe()
    )

# Outside
ndvi_out_mean_df = (
    ndvi_out#.sel(date=ndvi_out['date'].dt.month.isin([7]))
    .groupby('date.year')
    .mean(...)
    .to_dataframe()
    )

In [7]:
# Check to make sure things look ok

ndvi_out_mean_df

,NDVI,spatial_ref
year,,
2001,91.437182,0
2002,-26.842936,0
2003,-8.886499,0
2004,-54.362362,0
2005,135.748269,0
2006,24.021443,0
2007,-109.230466,0
2008,35.984796,0
2009,-70.890690,0


In [8]:
# Join the inside and outside DFs

ndvi_df = (ndvi_in_mean_df[['NDVI']]
           .join(ndvi_out_mean_df[['NDVI']], 
                 lsuffix=' Inside GRIC',
                 rsuffix=' Outside GRIC'))
ndvi_df

,NDVI Inside GRIC,NDVI Outside GRIC
year,,
2001,-798.066773,91.437182
2002,-893.761341,-26.842936
2003,-852.467468,-8.886499
2004,-901.557103,-54.362362
2005,-626.261012,135.748269
2006,-746.046521,24.021443
2007,-877.107961,-109.230466
2008,-788.169985,35.984796
2009,-888.534416,-70.890690


In [9]:
# Plot annual mean NDVI inside and outside on one plot

ndvi_df.hvplot(title='Mean July NDVI inside and outside Gila River Indian Community')

:NdOverlay   [Variable]
   :Curve   [year]   (value)

Now, take the difference between outside and inside the site boundary
and plot that. What do you observe? Don’t forget to write a headline and
description of your plot!

In [10]:
# Calculate the difference inside and outside
ndvi_df['Difference'] = ndvi_df['NDVI Inside GRIC'] - ndvi_df['NDVI Outside GRIC']
ndvi_df

,NDVI Inside GRIC,NDVI Outside GRIC,Difference
year,,,
2001,-798.066773,91.437182,-889.503955
2002,-893.761341,-26.842936,-866.918405
2003,-852.467468,-8.886499,-843.580969
2004,-901.557103,-54.362362,-847.194741
2005,-626.261012,135.748269,-762.009281
2006,-746.046521,24.021443,-770.067965
2007,-877.107961,-109.230466,-767.877495
2008,-788.169985,35.984796,-824.154781
2009,-888.534416,-70.890690,-817.643727


In [11]:
# Plot difference inside and outside the boundary

ndvi_df['Difference'].hvplot(title='Difference in NDVI inside GRIC vs outside')

:Curve   [year]   (Difference)

This plot shows the difference in NDVI inside the Gila River Indian Community and outside of it. There is generally an upward trend across the entire period, but a distinct uptick in the difference after 2012. Before 2012 there is significant variability year to year, while variability actually goes down after 2012. This clearly demonstrates the impact of returning water rights to the Gila River tribe.

# STEP -1: Wrap up

Don’t forget to store your variables so you can use them in other
notebooks! Replace `var1` and `var2` with the variable you want to save,
separated by spaces.

In [12]:
%store ndvi_da ndvi_

Stored 'ndvi_da' (Dataset)


UsageError: Unknown variable 'ndvi_'


Finally, be sure to `Restart` and `Run all` to make sure your notebook
works all the way through!